In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

import csv
import warnings
import importlib as imp
from timeit import default_timer
from scipy import stats
from scipy.interpolate import PchipInterpolator

import sys
sys.path.insert(0, "./forecaster/")
import mr_forecast as mr

import LMC
import archinfo

from oviraptor.utils import *
from oviraptor.constants import *
from oviraptor.chenkipping import *

In [2]:
#MAINPATH = 'C:/Users/djhoo/Documents/Oviraptor-master/'
MAINPATH = "/Users/research/projects/oviraptor/"
csv_file = MAINPATH + "Catalogs/oviraptor_B_manually_fixed.csv"

# number of bootstrap iterations
NBOOT = 1000

# mr_relation can be 'chen-kipping', 'ning-et-al', 'neil-rogers'
MR_RELATION = "chen-kipping"

# Define System class

In [3]:
class System:
    def __init__(self, sysid=None):
        self.sysid = sysid

# Read in the data

In [4]:
# here's the planetary system data from the archive
keys, vals = read_csv_file(csv_file, k_index=0, v_index=1)

data = {}
for k in keys:
    data[k] = np.array(get_csv_data(k, keys, vals))
    
nobj = len(data["pl_name"])
nsys = len(np.unique(data["hostname"]))

print("Loaded data for {0} planets in {1} systems".format(nobj, nsys))

Loaded data for 3903 planets in 2858 systems


In [5]:
# convert dictionary string lists to numerical arrays
int_keys = ["sy_snum", "sy_pnum", "disc_year", "pl_controv_flag", "cb_flag", "ttv_flag"]

for k in data.keys():
    if np.isin(k, int_keys):
        dtype = "int"
    else:
        dtype = "float"
    
    try:
        data[k] = get_num_vals_from_str(data[k], dtype=dtype)
    except:
        pass

# Do some cleanup

### Remove single-planet systems

In [6]:
keep = data["sy_pnum"] > 1

for k in data.keys():
    data[k] = data[k][keep]
    
nobj = len(data["pl_name"])
nsys = len(np.unique(data["hostname"]))

print("After eliminating single-planet systems, {0} planets in {1} systems remain".format(nobj, nsys))

After eliminating single-planet systems, 1722 planets in 677 systems remain


### Set nominal detection thresholds

In [7]:
rprs_cutoff = np.min(data["pl_ratror"][data["discoverymethod"] == "Transit"])
kamp_cutoff = np.min(data["pl_rvamp"][data["discoverymethod"] == "Radial Velocity"])

max_accept_mass = 4132
max_accept_rad = 11.1

### Remove giant planets - $m_p/M_{\oplus} > 4132$ or $r_p/R_{\oplus} > 11.1$

In [8]:
keep = (data["pl_bmasse"] < max_accept_mass)*(data["system_type"] == "Radial Velocity")
keep += (data["pl_rade"] < max_accept_rad)*(data["system_type"] == "Transit")
keep += (data["pl_bmasse"] < max_accept_mass)*(data["pl_rade"] < max_accept_rad)*(data["system_type"] == "Mixed")


for k in data.keys():
    data[k] = data[k][keep]
    
nobj = len(data["pl_name"])
nsys = len(np.unique(data["hostname"]))

print("After eliminating giant planets, {0} planets in {1} systems remain".format(nobj, nsys))

After eliminating giant planets, 1667 planets in 665 systems remain


### Print out some statistics on planet counts

In [9]:
RV = data["system_type"] == "Radial Velocity"
TR = data["system_type"] == "Transit"
MX = data["system_type"] == "Mixed"
npl = np.array(data["sy_pnum"], dtype="int")


# RADIAL VELOCITY
print("\nRADIAL VELOCITY")

nobj, nsys = len(data["pl_name"][RV]), len(np.unique(data["hostname"][RV]))
print(" {0} planets in {1} systems".format(nobj, nsys))

nobj, nsys = len(data["pl_name"][RV*(npl >= 3)]), len(np.unique(data["hostname"][RV*(npl >= 3)]))
print(" {0} planets in {1} high multiplicity (N >= 3) systems".format(nobj, nsys))


# TRANSIT
print("\nTRANSIT")

nobj, nsys = len(data["pl_name"][TR]), len(np.unique(data["hostname"][TR]))
print(" {0} planets in {1} systems".format(nobj, nsys))

nobj, nsys = len(data["pl_name"][TR*(npl >= 3)]), len(np.unique(data["hostname"][TR*(npl >= 3)]))
print(" {0} planets in {1} high multiplicity (N >= 3) systems".format(nobj, nsys))


# MIXED
print("\nMIXED")

nobj, nsys = len(data["pl_name"][MX]), len(np.unique(data["hostname"][MX]))
print(" {0} planets in {1} systems".format(nobj, nsys))

nobj, nsys = len(data["pl_name"][MX*(npl >= 3)]), len(np.unique(data["hostname"][MX*(npl >= 3)]))
print(" {0} planets in {1} high multiplicity (N >= 3) systems".format(nobj, nsys))


RADIAL VELOCITY
 270 planets in 106 systems
 137 planets in 38 high multiplicity (N >= 3) systems

TRANSIT
 1313 planets in 524 systems
 643 planets in 184 high multiplicity (N >= 3) systems

MIXED
 84 planets in 35 systems
 51 planets in 15 high multiplicity (N >= 3) systems


# Build lists of systems

In [10]:
rv_systems = []
tr_systems = []
mx_systems = []


for i, star in enumerate(np.unique(data["hostname"])):
    use = (data["hostname"] == star)
    
    s = System(sysid=star)
    s.planet_id = data["pl_name"][use]
    
    s.mstar = float(data["st_mass"][use][0])
    s.mstar_err = np.sqrt(data["st_masserr1"][use][0]**2 + data["st_masserr2"][use][0]**2)/np.sqrt(2)
    
    if s.mstar_err < 0.01:
        s.mstar_err = 0.01

    s.rstar = float(data["st_rad"][use][0])
    s.rstar_err = np.sqrt(data["st_raderr1"][use][0]**2 + data["st_raderr2"][use][0]**2)/np.sqrt(2)
    
    if s.rstar_err < 0.01:
        s.rstar_err = 0.01
        
    s.teff = int(data["st_teff"][use][0])
    s.teff_err = np.sqrt(data["st_tefferr1"][use][0]**2 + data["st_tefferr2"][use][0]**2)/np.sqrt(2)
    
    s.npl = np.sum(use)
    s.per = data["pl_orbper"][use]
    
    
    if data["system_type"][use][0] == "Radial Velocity":
        s.discoverymethod = "RV"
        
        s.kamp = data["pl_rvamp"][use]
        s.kamp_err = np.sqrt(data["pl_rvamperr1"][use]**2 + data["pl_rvamperr2"][use]**2)/np.sqrt(2)
        
        s.ecc = data["pl_orbeccen"][use]
        s.ecc_err = np.sqrt(data["pl_orbeccenerr1"][use]**2 + data["pl_orbeccenerr2"][use]**2)/np.sqrt(2)
        
        bad = np.isnan(s.ecc) + np.isnan(s.ecc_err)
        s.ecc[bad] = 0.0
        s.ecc_err[bad] = 0.0
        
        rv_systems.append(s)
        
        
    if data["system_type"][use][0] == "Transit":
        s.discoverymethod = "Transit"
        
        s.rprs = data["pl_ratror"][use]
        s.rprs_err = np.sqrt(data["pl_ratrorerr1"][use]**2 + data["pl_ratrorerr2"][use]**2)/np.sqrt(2)
        
        s.ecc = np.zeros(s.npl)
        s.ecc_err = np.zeros(s.npl)
        
        tr_systems.append(s)
        
        
    if data["system_type"][use][0] == "Mixed":
        s.discoverymethod = "Mixed"
        s.ttv_flag = data["ttv_flag"][use]
        
        s.kamp = data["pl_rvamp"][use]
        s.kamp_err = np.sqrt(data["pl_rvamperr1"][use]**2 + data["pl_rvamperr2"][use]**2)/np.sqrt(2)
                
        s.rprs = data["pl_ratror"][use]
        s.rprs_err = np.sqrt(data["pl_ratrorerr1"][use]**2 + data["pl_ratrorerr2"][use]**2)/np.sqrt(2)
        
        s.mp = data["pl_bmasse"][use]
        s.mp_err = np.sqrt(data["pl_bmasseerr1"][use]**2 + data["pl_bmasseerr2"][use]**2)/np.sqrt(2)
        
        s.rp = data["pl_rade"][use]
        s.rp_err = np.sqrt(data["pl_radeerr1"][use]**2 + data["pl_radeerr2"][use]**2)/np.sqrt(2)
        
        s.ecc = np.zeros(s.npl)
        s.ecc_err = np.zeros(s.npl)
        
        mx_systems.append(s)

# Draw probabilistic masses and radii

#### RV Systems: $K, M_{\star} \rightarrow m_p$

In [11]:
for i, s in enumerate(rv_systems):
    print(s.sysid)

    s.prob_mstar = stats.truncnorm((0.07-s.mstar)/s.mstar_err, np.inf, 
                                   loc=s.mstar, scale=s.mstar_err).rvs(NBOOT)

    s.prob_rstar = stats.truncnorm((0.09-s.rstar)/s.rstar_err, np.inf, 
                                   loc=s.rstar, scale=s.rstar_err).rvs(NBOOT)

    s.prob_teff = stats.truncnorm((2400-s.teff)/s.teff_err, np.inf, 
                                   loc=s.teff, scale=s.teff_err).rvs(NBOOT)

    s.prob_kamp = np.zeros((NBOOT,s.npl))
    s.prob_rprs = np.ones((NBOOT,s.npl))*np.nan
    s.prob_mp = np.zeros((NBOOT,s.npl))
    s.prob_rp = np.ones((NBOOT,s.npl))*np.nan


    for j in range(s.npl):

        draw = np.ones(NBOOT, dtype="bool")
        count = 0

        while (np.sum(draw) > 0)*(count < 25):
            count +=1


            s.prob_kamp[:,j][draw] = stats.truncnorm((kamp_cutoff-s.kamp[j])/s.kamp_err[j], np.inf, 
                                                      loc=s.kamp[j], scale=s.kamp_err[j]).rvs(np.sum(draw))

            for n in range(NBOOT):
                if draw[n]:
                    s.prob_mp[n,j] = mp_from_kamp(s.prob_kamp[n,j], s.per[j], s.prob_mstar[n], s.ecc[j])

            draw = s.prob_mp[:,j] > max_accept_mass

        if np.sum(draw) > 0:
            print(" {0} draws failed to converge".format(np.sum(draw)))
            s.prob_mp[:,j][draw] = np.random.choice(s.prob_mp[:,j][~draw], size=np.sum(draw))
            s.prob_mp[:,j][draw] += np.random.normal(size=np.sum(draw))*0.001

24 Sex
47 UMa
55 Cnc
61 Vir
7 CMa
BD+20 2457
 885 draws failed to converge
BD-08 2823
BD-11 4672
DMPP-1
GJ 1061
GJ 1148
GJ 163
GJ 180
GJ 3138
GJ 317
GJ 3293
GJ 3998
GJ 414 A
GJ 433
GJ 581
GJ 676 A
GJ 687
GJ 876
GJ 887
GJ 9066
HD 10180
HD 102329
HD 108874
HD 109271
HD 113538
HD 114783
HD 116029
HD 11964
HD 125612
HD 12661
HD 128311
HD 134987
HD 136352
HD 13808
HD 13908
HD 141399
HD 142
HD 147018
HD 147873
HD 154857
HD 155358
HD 156279
HD 158259
HD 159243
HD 160691
HD 164922
HD 168443
HD 176986
HD 177830
HD 181433
HD 183263
HD 187123
HD 190360
HD 202696
HD 204313
HD 20781
HD 20794
HD 215152
HD 215497
HD 216520
HD 217107
HD 219134
HD 27894
HD 30177
HD 31527
HD 33142
HD 33844
HD 34445
HD 37124
HD 40307
HD 4203
HD 45364
HD 47186
HD 4732
HD 47366
HD 50499
HD 60532
HD 67087
HD 69830
HD 73526
HD 74156
HD 7924
HD 82943
HD 92788
HD 99706
HIP 14810
HIP 38594
HIP 5158
HIP 54373
HIP 57274
HIP 65407
HIP 67851
Pr0211
TYC 1422-614-1
 3 draws failed to converge
Teegarden's Star
Wolf 1061
XO-2 S
YZ Cet


### Transit Systems: $r_p/R_{\star}, R_{\star} \rightarrow r_p$

In [12]:
for i, s in enumerate(tr_systems):
    print(s.sysid)

    s.prob_mstar = stats.truncnorm((0.07-s.mstar)/s.mstar_err, np.inf, 
                                   loc=s.mstar, scale=s.mstar_err).rvs(NBOOT)

    s.prob_rstar = stats.truncnorm((0.09-s.rstar)/s.rstar_err, np.inf, 
                                   loc=s.rstar, scale=s.rstar_err).rvs(NBOOT)

    s.prob_teff = stats.truncnorm((2400-s.teff)/s.teff_err, np.inf, 
                                   loc=s.teff, scale=s.teff_err).rvs(NBOOT)

    s.prob_kamp = np.ones((NBOOT,s.npl))*np.nan
    s.prob_rprs = np.zeros((NBOOT,s.npl))
    s.prob_rp = np.zeros((NBOOT,s.npl))
    s.prob_mp = np.ones((NBOOT,s.npl))*np.nan


    for j in range(s.npl):

        draw = np.ones(NBOOT, dtype="bool")
        count = 0

        while (np.sum(draw) > 0)*(count < 10):
            count +=1

            s.prob_rprs[:,j][draw] = stats.truncnorm((rprs_cutoff-s.rprs[j])/s.rprs_err[j], np.inf, 
                                                      loc=s.rprs[j], scale=s.rprs_err[j]).rvs(np.sum(draw))

            s.prob_rp[:,j][draw] = s.prob_rprs[:,j][draw]*s.prob_rstar[draw]*RSUN/REARTH

            draw = s.prob_rp[:,j] > max_accept_rad


        if np.sum(draw) > 0:
            print(" {0} draws failed to converge".format(np.sum(draw)))
            s.prob_rp[:,j][draw] = np.random.choice(s.prob_rp[:,j][~draw], size=np.sum(draw))
            s.prob_rp[:,j][draw] += np.random.normal(size=np.sum(draw))*0.001

CoRoT-24
EPIC 212737443
EPIC 220674823
EPIC 249893012
GJ 143
GJ 9827
HD 106315
HD 108236
HD 15337
HD 191939
HD 23472
HD 3167
HD 63433
HIP 41378
HR 858
K2-117
K2-133
K2-136
K2-138
K2-141
K2-146
K2-148
K2-154
K2-155
K2-158
K2-16
K2-165
K2-166
K2-168
K2-170
K2-172
K2-183
 11 draws failed to converge
K2-187
K2-188
K2-189
K2-19
K2-190
K2-195
K2-198
K2-199
K2-201
K2-21
K2-219
K2-223
K2-224
K2-229
K2-233
K2-239
K2-24
K2-240
K2-243
K2-247
K2-254
K2-264
K2-266
K2-268
K2-270
K2-275
K2-282
K2-285
K2-290
K2-3
K2-316
K2-32
K2-35
K2-36
K2-37
K2-38
K2-43
K2-5
K2-50
K2-58
K2-59
K2-62
K2-63
K2-72
K2-75
K2-8
K2-80
K2-83
K2-84
K2-90
KIC 5437945
 1 draws failed to converge
KIC 9663113
KOI-1783
KOI-351
KOI-7892
KOI-94
Kepler-10
Kepler-100
Kepler-1001
Kepler-101
Kepler-1016
Kepler-102
Kepler-103
Kepler-104
Kepler-1047
Kepler-105
Kepler-1050
Kepler-106
Kepler-1065
Kepler-107
Kepler-1073
Kepler-108
Kepler-1085
Kepler-1086
Kepler-109
Kepler-1093
Kepler-11
Kepler-110
Kepler-111
Kepler-112
Kepler-1129
Kepler-113

### Mixed Systems

In [13]:
for i, s in enumerate(mx_systems):
    
    print(s.sysid)

    s.prob_mstar = stats.truncnorm((0.07-s.mstar)/s.mstar_err, np.inf, 
                                   loc=s.mstar, scale=s.mstar_err).rvs(NBOOT)

    s.prob_rstar = stats.truncnorm((0.09-s.rstar)/s.rstar_err, np.inf, 
                                   loc=s.rstar, scale=s.rstar_err).rvs(NBOOT)

    s.prob_teff = stats.truncnorm((2400-s.teff)/s.teff_err, np.inf, 
                                   loc=s.teff, scale=s.teff_err).rvs(NBOOT)

    s.prob_kamp = np.zeros((NBOOT,s.npl))
    s.prob_rprs = np.zeros((NBOOT,s.npl))
    s.prob_mp = np.zeros((NBOOT,s.npl))
    s.prob_rp = np.zeros((NBOOT,s.npl))

    for j in range(s.npl):

        # try to calculate mass from RV semi-amplitude
        draw = np.ones(NBOOT, dtype="bool")
        count = 0

        while (np.sum(draw) > 0)*(count < 25):
            count +=1

            if ~np.isnan(s.kamp[j]/s.kamp_err[j]):
                s.prob_kamp[:,j][draw] = stats.truncnorm((kamp_cutoff-s.kamp[j])/s.kamp_err[j], np.inf, 
                                                          loc=s.kamp[j], scale=s.kamp_err[j]).rvs(np.sum(draw))

                for n in range(NBOOT):
                    if draw[n]:
                        s.prob_mp[n,j] = mp_from_kamp(s.prob_kamp[n,j], s.per[j], s.prob_mstar[n])

                draw = s.prob_mp[:,j] > max_accept_mass


        # try to calculate radius from rp/Rs
        draw = np.ones(NBOOT, dtype="bool")
        count = 0

        while (np.sum(draw) > 0)*(count < 25):
            count +=1
            if ~np.isnan(s.rprs[j]/s.rprs_err[j]):
                s.prob_rprs[:,j][draw] = stats.truncnorm((rprs_cutoff-s.rprs[j])/s.rprs_err[j], np.inf, 
                                                          loc=s.rprs[j], scale=s.rprs_err[j]).rvs(np.sum(draw))

                s.prob_rp[:,j][draw] = s.prob_rprs[:,j][draw]*s.prob_rstar[draw]*RSUN/REARTH

                draw = s.prob_rp[:,j] > max_accept_rad


        # if RV amplitude is not available, try drawing mass directly (might be available from TTVs)
        if np.all(s.prob_mp[:,j]==0)*s.ttv_flag[j]:
            draw = np.ones(NBOOT, dtype="bool")
            count = 0

            while (np.sum(draw) > 0)*(count < 25):
                count +=1

                if ~np.isnan(s.mp[j]/s.mp_err[j]):
                    s.prob_mp[:,j] = stats.truncnorm(-s.mp[j]/s.mp_err[j], np.inf,
                                                     loc=s.mp[j], scale=s.mp_err[j]).rvs(np.sum(draw))

                    for n in range(NBOOT):
                        if draw[n]:
                            s.prob_kamp[n,j] = kamp_from_mp(s.per[j], s.prob_mp[n,j], s.prob_mstar[n])

                    draw = (s.prob_mp[:,j] > max_accept_mass) + (s.prob_kamp[:,j] < kamp_cutoff)


        if np.all(s.prob_kamp[:,j] == 0):
            s.prob_kamp[:,j] = np.nan
        if np.all(s.prob_rprs[:,j] == 0):
            s.prob_rprs[:,j] = np.nan
        if np.all(s.prob_mp[:,j] == 0):
            s.prob_mp[:,j] = np.nan
        if np.all(s.prob_rp[:,j] == 0):
            s.prob_rp[:,j] = np.nan
            
        if np.all(s.prob_mp[:,j]==np.nan)*np.all(s.prob_rp[:,j]==np.nan):
            raise ValueError("Both mass and radius arrays returned all NaN values...something has gone wrong")

CoRoT-20
CoRoT-7
GJ 1132
GJ 3473
GJ 357
HAT-P-11
HD 110113
HD 213885
HD 39091
HD 5278
HD 86226
K2-111
KOI-142
Kepler-122
Kepler-160
Kepler-19
Kepler-20
Kepler-25
Kepler-338
Kepler-37
Kepler-407
Kepler-411
Kepler-424
Kepler-454
Kepler-46
Kepler-48
Kepler-539
Kepler-56
Kepler-65
Kepler-68
Kepler-82
Kepler-94
WASP-107
WASP-148
WASP-47


# Save probabilistic draws

In [14]:
all_systems = rv_systems + tr_systems + mx_systems

draws = dict.fromkeys(["planet_id", "mstar", "rstar", "teff", "kamp", "rprs", "mp", "rp", "period", "ecc"])

for k in draws.keys():
    draws[k] = []
     
for i, s in enumerate(all_systems):
    for j in range(s.npl):
        draws["planet_id"].append(s.planet_id[j])
        draws["mstar"].append(s.prob_mstar)
        draws["rstar"].append(s.prob_rstar)
        draws["teff"].append(s.prob_teff)
        draws["kamp"].append(s.prob_kamp[:,j])
        draws["rprs"].append(s.prob_rprs[:,j])
        draws["mp"].append(s.prob_mp[:,j])
        draws["rp"].append(s.prob_rp[:,j])
        draws["period"].append(s.per[j])
        draws["ecc"].append(s.ecc[j])
        
nobj = len(draws["planet_id"])

In [15]:
probabilistic_keys = ["mstar", "rstar", "teff", "kamp", "rprs", "mp", "rp"]
deterministic_keys = ["period", "ecc"]


for key in probabilistic_keys:
    filepath = MAINPATH + "Catalogs/mr_draws/" + key + ".csv"

    with open(filepath, "w") as outfile:
        writer = csv.writer(outfile)

        header = ["planet_id", key + "_mean"]
        for n in range(NBOOT):
            header = np.concatenate((header, ["{:s}_{:03d}".format(key, n)]))
        writer.writerow(header)

        for i in range(nobj):
            arr = np.concatenate(([draws["planet_id"][i]], [np.mean(draws[key][i])], draws[key][i]))
            writer.writerow(arr)
            
            
for key in deterministic_keys:
    filepath = MAINPATH + "Catalogs/mr_draws/" + key + ".csv"

    with open(filepath, "w") as outfile:
        writer = csv.writer(outfile)

        header = ["planet_id", key]
        writer.writerow(header)
        
        for i in range(nobj):
            arr = np.concatenate(([draws["planet_id"][i]], [draws[key][i]]))
            writer.writerow(arr)